In [22]:
import argparse
import math
import os

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

from build_vocab import WordVocab
from dataset import Seq2seqDataset

PAD = 0
UNK = 1
EOS = 2
SOS = 3
MASK = 4

class PositionalEncoding(nn.Module):
    "Implement the PE function. No batch support?"
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model) # (T,H)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

class TrfmSeq2seq(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, n_layers, dropout=0.1):
        super(TrfmSeq2seq, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(in_size, hidden_size)
        self.pe = PositionalEncoding(hidden_size, dropout)
        self.trfm = nn.Transformer(d_model=hidden_size, nhead=4, 
        num_encoder_layers=n_layers, num_decoder_layers=n_layers, dim_feedforward=hidden_size)
        self.out = nn.Linear(hidden_size, out_size)

    def forward(self, src):
        # src: (T,B)
        embedded = self.embed(src)  # (T,B,H)
        embedded = self.pe(embedded) # (T,B,H)
        hidden = self.trfm(embedded, embedded) # (T,B,H)
        out = self.out(hidden) # (T,B,V)
        out = F.log_softmax(out, dim=2) # (T,B,V)
        return out # (T,B,V)

    def _encode(self, src):
        # src: (T,B)
        embedded = self.embed(src)  # (T,B,H)
        embedded = self.pe(embedded) # (T,B,H)
        output = embedded
        for i in range(self.trfm.encoder.num_layers - 1):
            output = self.trfm.encoder.layers[i](output, None)  # (T,B,H)
        penul = output.detach().numpy()
        output = self.trfm.encoder.layers[-1](output, None)  # (T,B,H)
        if self.trfm.encoder.norm:
            output = self.trfm.encoder.norm(output) # (T,B,H)
        output = output.detach().numpy()
        # mean, max, first*2
        return np.hstack([np.mean(output, axis=0), np.max(output, axis=0), output[0,:,:], penul[0,:,:] ]) # (B,4H)
    
    def encode(self, src):
        # src: (T,B)
        batch_size = src.shape[1]
        if batch_size<=100:
            return self._encode(src)
        else: # Batch is too large to load
            print('There are {:d} molecules. It will take a little time.'.format(batch_size))
            st,ed = 0,100
            out = self._encode(src[:,st:ed]) # (B,4H)
            while ed<batch_size:
                st += 100
                ed += 100
                out = np.concatenate([out, self._encode(src[:,st:ed])], axis=0)
            return out

#     return parser.parse_args()



def evaluate(model, test_loader, vocab):
    model.eval()
    total_loss = 0
    total_acc = 0
    for b, sm in enumerate(test_loader):
        sm = torch.t(sm.cuda()) # (T,B)
        with torch.no_grad():
            output = model(sm) # (T,B,V)
        
        acc = 0.#torch.eq(torch.sum(torch.eq(torch.max(output, dim=2).values, sm), dim=0), sm.shape[0]).sum()
        
        loss = F.nll_loss(output.view(-1, len(vocab)),
                               sm.contiguous().view(-1),
                               ignore_index=PAD)
        total_loss += loss.item()
        total_acc += acc
    return total_loss / len(test_loader), total_acc / len(test_loader)

In [20]:
from collections import namedtuple
Args = namedtuple('Args', ['n_epoch', 'vocab', 'data', 'outdir', 'name', 'seq_len', 'batch_size', 'n_worker', 'hidden',\
                         'n_layer', 'n_head', 'lr', 'hpu'])

args = Args(5, 'vocab.pkl', 'my_train_smiles.csv','result','ST',220,8,4,256,4,4,1e-4,0)

In [23]:
assert torch.cuda.is_available()

print('Loading dataset...')
vocab = WordVocab.load_vocab(args.vocab)
dataset = Seq2seqDataset(pd.read_csv(args.data)['first'].values, vocab)
test_size = 50000
train, test = torch.utils.data.random_split(dataset, [len(dataset)-test_size, test_size])
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True, num_workers=args.n_worker)
test_loader = DataLoader(test, batch_size=args.batch_size, shuffle=False, num_workers=args.n_worker)
print('Train size:', len(train))
print('Test size:', len(test))
del dataset, train, test

model = TrfmSeq2seq(len(vocab), args.hidden, len(vocab), args.n_layer).cuda()

model.load_state_dict(torch.load(os.path.normpath('trfm_new_1_120000.pkl')))


optimizer = optim.Adam(model.parameters(), lr=args.lr)
print(model)
print('Total parameters:', sum(p.numel() for p in model.parameters()))

best_loss = None
for e in range(1, 2):#args.n_epoch):
    for b, sm in tqdm(enumerate(train_loader)):
        sm = torch.t(sm.cuda()) # (T,B)
        optimizer.zero_grad()
        output = model(sm) # (T,B,V)
        loss = F.nll_loss(output.view(-1, len(vocab)),
                sm.contiguous().view(-1), ignore_index=PAD)
        loss.backward()
        optimizer.step()
        if b%10==0:
            print('Train {:3d}: iter {:5d} | loss {:.3f} | ppl {:.3f}'.format(e, b, loss.item(), math.exp(loss.item())))
        if b%100==0:
            loss,acc = evaluate(model, test_loader, vocab)
            print('Val {:3d}: iter {:5d} | loss {:.3f} | ppl {:.3f}'.format(e, b, loss, math.exp(loss)))
            print('acc: ', acc)
            # Save the model if the validation loss is the best we've seen so far.
            if not best_loss or loss < best_loss:
                print("[!] saving model...")
                if not os.path.isdir(".save"):
                    os.makedirs(".save")
                torch.save(model.state_dict(), './.save/trfm_new_%d_%d.pkl' % (e,b))
                best_loss = loss

Loading dataset...
Train size: 1672298
Test size: 50000
TrfmSeq2seq(
  (embed): Embedding(45, 256)
  (pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (trfm): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_pr

0it [00:00, ?it/s]

Train   1: iter     0 | loss 1.893 | ppl 6.636
Val   1: iter     0 | loss 1.877 | ppl 6.532
acc:  0.0
[!] saving model...


11it [12:10,  6.50s/it]

Train   1: iter    10 | loss 1.843 | ppl 6.314


...

Train   1: iter    60 | loss 0.015 | ppl 1.015


71it [12:30,  5.29it/s]

Train   1: iter    70 | loss 0.003 | ppl 1.003


75it [12:31, 10.03s/it]


KeyboardInterrupt: 

In [13]:
import torch

output = torch.zeros(2,3,5) + torch.ge(torch.randn(2,3,5), torch.as_tensor(0.5))
print(output)
sm = torch.ones(2,3)
print(sm)
torch.eq(torch.sum(torch.eq(torch.max(output, dim=2).values, sm), dim=0), sm.shape[0]).sum()

tensor([[[0., 1., 0., 1., 0.],
         [0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 1.]],

        [[0., 1., 1., 1., 1.],
         [1., 1., 0., 0., 0.],
         [1., 0., 0., 1., 1.]]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


tensor(2)